In [ ]:
import pandas as pd
import os
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("../Draft_Machine/nba_data/2018.csv")

files = [file for file in os.listdir('./Nba_Data')]

all_year_data = pd.DataFrame()

for file in files:
    df = pd.read_csv("Nba_Data/" + file)
    all_year_data = pd.concat([all_year_data, df])
    
all_year_data.head()

In [ ]:
all_year_data.shape

In [ ]:
new_header = all_year_data.iloc[0] #grab the first row for the header
New_all_year_data = all_year_data[1:] #take the data less the header row
New_all_year_data.columns = new_header #set the header row as the df header
New_all_year_data.head(200)
                      

In [ ]:
df =New_all_year_data.drop([0])
df.head(63)

In [ ]:
df_sort = df.sort_values(by = "Year")
df_sort.head()

In [ ]:
college_df=df_sort.dropna(axis=0, subset=["College"], inplace=False)
college_df.head(10)

In [ ]:
clean=college_df.drop(columns=["FG%", "TRB", "AST", "3P%","FT%","TRB","AST", "Pk"])
clean.head()

In [ ]:
clean.columns=["Win Shares", "Mins Played (per game)", "Player","Games","Year","Rank","Points(per game)","WS/48", "Box +/-","Value Overall Replacement Player", "Team", "College", "Years","Mins Played (Total)","Points (Total)"]
                                
clean.rename(columns={df_sort.columns[0]:"Win Shares"})


In [ ]:
df_reindex=clean.reset_index()
df_reindex=df_reindex.drop(['index'],axis=1)
df_reindex.head(20)

In [ ]:
clean_df=df_reindex.fillna(value=0)
clean_df.head()

In [ ]:
clean_df.dtypes

In [ ]:
clean_df["Win Shares"]= clean_df["Win Shares"].astype ('float')
clean_df["Mins Played (per game)"]=clean_df["Mins Played (per game)"].astype ('float')
clean_df["Points(per game)"]=clean_df["Points(per game)"].astype ('float')
clean_df["WS/48"]=clean_df["WS/48"].astype ('float')
clean_df["Box +/-"]=clean_df["Box +/-"].astype ('float')
clean_df["Value Overall Replacement Player"]=clean_df["Value Overall Replacement Player"].astype ('float')


clean_df["Games"]=clean_df["Games"].astype("int")
clean_df["Year"]=clean_df["Year"].astype("int")
clean_df["Rank"]=clean_df["Rank"].astype("int")
clean_df["Years"]=clean_df["Years"].astype("int")
clean_df["Mins Played (Total)"]=clean_df["Mins Played (Total)"].astype("int")
clean_df["Points (Total)"]=clean_df["Points (Total)"].astype("int")

clean_df["Player"]=clean_df["Player"].astype(str)
#  "Player":str,

#  "Team":str, 
# "College":str, 


In [ ]:
clean_df.dtypes

In [ ]:
clean_df.info()

In [ ]:
#to calculate median w/s of NBA player remove draft picks who did not play 
no_zeros = clean_df["Games"] != 0
no_zero_df = clean_df[no_zeros]
len(no_zero_df)

In [ ]:
#we will use the mean to separate above-average from below-average players
print(no_zero_df['WS/48'].median())

In [ ]:
#create blank column for Class of player
clean_df["Class"] =''

In [ ]:
#run for loop to assingn Star, Above Avg, Below Avg of Bust to the Class column
for i in range(len(clean_df)) : 
    if clean_df.loc[i,'Year'] < 2018 and clean_df.loc[i,'Years'] <= 2:
        clean_df.loc[i,'Class'] = "Bust"
    elif clean_df.loc[i,'Year'] == 2018 and clean_df.loc[i,'Years'] < 2:
        clean_df.loc[i,'Class'] = "Bust"
    elif clean_df.loc[i,'Year'] == 2019 and clean_df.loc[i,'Years'] < 1:
        clean_df.loc[i,'Class'] = "Bust"
    elif clean_df.loc[i,'Mins Played (per game)'] > 15 and clean_df.loc[i,'WS/48'] >= .12:
        clean_df.loc[i,'Class'] = "Star"
    elif clean_df.loc[i,'Mins Played (per game)'] > 15 and clean_df.loc[i,'WS/48'] < .12 and clean_df.loc[i,'WS/48'] > .068:
        clean_df.loc[i,'Class'] = "Above Avg"
    else:
        clean_df.loc[i,'Class']  = "Below Avg"

In [ ]:
#check df
clean_df.head(10)

In [ ]:
#count number of players in each class
type_df = clean_df.groupby('Class').count()
type_df

In [ ]:
#check percentage of Classes
total = type_df["Games"].sum()
above = (type_df.iloc[0, 0]/total)*100
below = (type_df.iloc[1, 0]/total)*100
bust = (type_df.iloc[2, 0]/total)*100
star = (type_df.iloc[3, 0]/total)*100
print(f'% of Above Avg Players: {above}')
print(f'% of Below Avg Players: {below}')
print(f'% of Bust Players: {bust}')
print(f'% of Star Players: {star}')

In [ ]:
# # #df.to_csv(file_name, sep='\t', encoding='utf-8')
clean_df.to_csv("NBA_data.csv",index = False, header = True)
